In [ ]:
from scholarly import scholarly
import json
import os

# Search for the author
search_query = scholarly.search_author_id("WpXJ758AAAAJ")
author = scholarly.fill((search_query))

journals = []
conferences = []
for pub in author.get('publications', []):
    detailed = scholarly.fill(pub)
    
    
    if detailed.get('bib', {}).get('conference', None):
        conf = detailed.get('bib', {})
        conf_text = (conf.get('title', '') + ', ' + conf.get('conference', '') + ', ' + conf.get('author', '') + ', ' + str(conf.get('pub_year', '')))
        conferences.append(conf_text)
    if detailed.get('bib', {}).get('journal', None):
        journal = detailed.get('bib', {})
        journal_text = (journal.get('title', '') + ', ' + conf.get('journal', '') + ', ' + conf.get('author', '') + ', ' + str(conf.get('pub_year', '')))
        journals.append(journal_text)
        



In [20]:
conference_dicts = [{"title": c} for c in conferences]
with open("_data/publicationss.json", "w", encoding="utf-8") as f:
	json.dump(conference_dicts, f, ensure_ascii=False, indent=4)
 
journal_dicts = [{"title": c} for c in journals]
with open("_data/publications.json", "w", encoding="utf-8") as f:
	json.dump(journal_dicts, f, ensure_ascii=False, indent=4)

In [ ]:
from scholarly import scholarly
import json

def fetch_author_publications(id):
    author = scholarly.fill((scholarly.search_author_id(id)))
    result = []
    for pub in author.get('publications', []):
        detailed = scholarly.fill(pub)
        bib = detailed.get('bib', {})
        item = {
            "id": """http://zotero.org/groups/...""",  # you may generate or leave placeholder
            "type": "article-journal",               # adjust based on content
            "title": bib.get("title"),
            "container-title": bib.get("venue"),
            "page": bib.get("pages"),
            "volume": bib.get("volume"),
            "issue": bib.get("issue"),
            "source": "Google Scholar",
            "abstract": detailed.get("abstract"),
            "DOI": bib.get("doi"),
            "author": bib.get("author", []),
            "issued": {"date-parts": [[bib.get("pub_year")]]}
        }
        result.append(item)
        with open(f"./_data/journals.json", "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=4)

print(fetch_author_publications("H4gjHdYAAAAJ"))


KeyboardInterrupt: 